In [ ]:
# You are going to need to run this cell, restart the runtime after running this command,
# then start over before you can run the code in this notebook.
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz


     |████████████████████████████████| 96.4 MB 95 kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-py3-none-any.whl size=98072934 sha256=e0dbf41bc9add21d8a89ee9e8ecd92557e6fab7644d0d81abcad5c77856be286
  Stored in directory: /root/.cache/pip/wheels/d1/e8/f9/fd0d324ec96ab55e0a8ae395183c22260a4eeaf166ade7ad22
Successfully built en-core-web-md


In [ ]:
import sklearn
estimators = sklearn.utils.all_estimators(type_filter=None)
for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)


AdaBoostClassifier
BaggingClassifier
BayesianGaussianMixture
BernoulliNB
CalibratedClassifierCV
CategoricalNB
ClassifierChain
ComplementNB
DecisionTreeClassifier
DummyClassifier
ExtraTreeClassifier
ExtraTreesClassifier
GaussianMixture
GaussianNB
GaussianProcessClassifier
GradientBoostingClassifier
GridSearchCV
HalvingGridSearchCV
HalvingRandomSearchCV
HistGradientBoostingClassifier
KNeighborsClassifier
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LogisticRegression
LogisticRegressionCV
MLPClassifier
MultiOutputClassifier
MultinomialNB
NuSVC
OneVsRestClassifier
Pipeline
QuadraticDiscriminantAnalysis
RFE
RFECV
RadiusNeighborsClassifier
RandomForestClassifier
RandomizedSearchCV
SGDClassifier
SVC
SelfTrainingClassifier
StackingClassifier
VotingClassifier


In [ ]:
import pandas as pd
import math
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import dates
from datetime import datetime
import re
import calendar
import json

from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, GaussianNB, MultinomialNB 
from sklearn import tree
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import spacy
from spacy.lang.en import English

import jieba
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install textstat

     |████████████████████████████████| 105 kB 13.1 MB/s 
     |████████████████████████████████| 2.0 MB 35.6 MB/s 


# Upload your data
To upload your own files for access in Google Colab:

1. Save the file in Google Drive.

2. Once it's saved, find it in your drive, right-click, and click `Get Shareable Link`.

3. Flip the toggle in the pop-up to `Link Sharing On`, then click `Sharing Settings`.

4. In the dropdown menu, choose `Anyone with the link can view`, then click `Copy Link`.

5. Paste that link in the code field below. 

6. Alter the link to the format below. Specifically, the link provided by Google Drive is written by default as:

`https://docs.google.com/document/d/ABCDEFG`

Replace this with:

`https://docs.google.com/uc?export=download&id=ABCDEFG`

Make sure that the unique identifier (`ABCDEFG`) above is identical to what you copied from Google Drive. Once you have saved the file to the Google Colab server, you can now open that file in Python, as in the next cell below.


In [ ]:
#https://drive.google.com/file/d/1gAbdQaJOhjqLIfubpvCjG399TroHGvsg/view?usp=sharing

In [ ]:
!gdown --id 1gAbdQaJOhjqLIfubpvCjG399TroHGvsg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1gAbdQaJOhjqLIfubpvCjG399TroHGvsg
To: /content/training_labeled.csv
100% 30.2k/30.2k [00:00<00:00, 11.3MB/s]


# Homework 5 Part Two

Last week you annotated a set of at least 200 microblog posts (your Training Set) with a supervised learning task designed by you and your partner, with your own annotation manual. 

By now, you should have two columns of labels for each post, and you should know the approximate upper bound of machine learning performance on this data, equal to your inter-rater reliability.

Save your work to a CSV file with the two annotator's labels (name the columns `labels_A` and `labels_B`), and a third column with the original text of the annotated post. 



# Task 2.1

Divide your data into a training set and a test set made up of 20% of the data. If you have 200 rows, your training set should have 160 examples, and your test set should have 40 rows.

In [ ]:
# create dataframe for the dataset
df=pd.read_csv("training_labeled.csv")
df=df.drop('Unnamed: 0', axis=1)
df=df.dropna()
df=df.loc[:,["text","labels_A","labels_B"]]
df.head()

,text,labels_A,labels_B
0,&gt;Falling for the Onion picking John Cena fo...,Positive,Positive
1,Sunday morning at the First Church of George H...,Neutral,Positive
2,Paper Towns is released in the UK tomorrow and...,Negative,Negative
3,Remember when Pain was a just shadowy figure? ...,Neutral,Neutral
4,Tomorrow Shawn is going to be an hour away fro...,Positive,Negative


In [ ]:
# divide the dataset to 80% train, 20% test. 
df_training=df.sample(frac=0.8, random_state=333)
df_testing=df.drop(df_training.index)
len(df_training)

160

# Task 2.2

Use 10-fold cross-validation on your training set. Each fold should contain 8% of the original training data (if you had 200 rows originally and use an 80% training set, each fold for optimization will contain 16 examples). 

Optimize a machine learning classifier predicting `labels_A`, using features you extract from the microblog texts. 

Choose and perform **THREE** of the following optimizations:

   - a) Compare Naïve Bayes, Logistic Regression, and SVMs on a unigram feature space.
   - b) Compare a unigram feature space with a feature space that also includes longer N-grams.
   - c) Compare a unigram feature space with a feature space that removes stopwords.
   - d) Vary the vocabulary size of your n-gram feature space and evaluate how performance changes.

If you are working with English data, additional options include:
   - e) Compare an n-gram feature space with a feature space that also includes part-of-speech n-grams.
   - f) Compare an n-gram feature space to a word embedding feature space.
   - g) Compare a standard n-gram feature space with a lemmatized feature space.
   - h) For the one most accurate model so far, tune your hyperparameters.
   - i) For Naive Bayes, evaluate different implementations: ComplementNB, MultinomialNB, BernoulliNB
   - j) For Support Vector Machines, evaluate different kernels including a polynomial kernel and a radial basis function kernel.
   - k) For Logistic Regression, try L1 and L2 regularization, as well as unregularized features.

Report the performance of your best-tuned model on the cross-validated training set.


In [ ]:
# useful functions from example notebook
"""
Options for noisy: "loud", "quiet", and other. Any other value prints nothing (silent).
"""

# Start by defining a function to evaluate a classifier's predictions
def evaluate(y_pred, y_actual, metrics, model_name = 'model'):
    # Compute Confusion Matrix
    conf_matrix = confusion_matrix(y_actual, y_pred)

    # Compute and store each metric
    model_metrics = {}
    for (metric_name, metric) in metrics.items():
        result = metric(y_actual, y_pred)
        model_metrics[metric_name] = result

    return conf_matrix, model_metrics

# Then define a function that trains a classifier and evaluates it on one fold
def evaluate_one_fold(classifier_name, classifier, X_train, y_train, X_test, y_test, metrics, fold_num, noisy = 'loud', labels=[]):

    # Train and Evaluate Model
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    conf_matrix, model_metrics = evaluate(y_pred, y_test, metrics, model_name = classifier_name)

    # Display Results appropriately when noisy is set to 'loud' or 'quiet'
    if noisy == 'quiet' and fold_num == 0:
        print(f"{classifier_name}: Fold {fold_num}", end = '')
    elif noisy == 'quiet':
        print(f'...{fold_num}', end ='')
    elif noisy == 'loud':
        print(f"{classifier_name}: Fold {fold_num} Results")
        ConfusionMatrixDisplay(conf_matrix, labels).plot(values_format='.4g')
        plt.show()
        print(model_metrics)
        print("------------------------")

    return model_metrics

# Then define a function to evaluate over all folds
def evaluate_all_folds(classifier_name, classifier, X, y, kf, metrics, noisy = 'loud', labels=[]):

    # Initialize tracking variables
    all_fold_metrics = {metric_name: [] for metric_name in metrics}

    # Iterate over each fold
    for fold_num, (train_index, test_index) in enumerate(kf.split(X, y)):
        # Get the data subset for the current fold
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        # Train and Evaluate the Model
        model_metrics =  evaluate_one_fold(classifier_name, classifier, X_train, y_train, X_test, y_test, metrics, fold_num, noisy, labels=labels)

        # Update our tracking variables
        [all_fold_metrics[metric_name].append(metric_val) for metric_name, metric_val in model_metrics.items()]

    return all_fold_metrics

#Dont worry about these two lines
@ignore_warnings(category=ConvergenceWarning)
@ignore_warnings(category=UndefinedMetricWarning)
# Then define a function to compare different classifiers
def compare_classifiers(classifiers, metrics, metric_to_optimize, df, feature_set,
                        target, folds = 10, shuffle = True, noisy='loud', labels=[]):
    # Initialize tracking variables
    best = 0
    best_name = None
    classifier_comparison = {}

    # Set up dataset and cross validation
    X = df.loc[:, feature_set]
    X = pd.get_dummies(X)
    y = df[target]
    kf = StratifiedKFold(n_splits=folds, shuffle=shuffle, random_state=123)


    # For each classifier
    for classifier_name, classifier in classifiers.items():
        # Evaluate on all metrics for all folds
        all_fold_metrics = evaluate_all_folds(classifier_name, classifier, X, y, kf, metrics, noisy = noisy, labels=labels)

        # Compute average performance on metric to optimize over
        optimization_metric_avg = np.mean(all_fold_metrics[metric_to_optimize])

        # Update Tracking Variables
        if optimization_metric_avg > best:
            best = optimization_metric_avg
            best_name = classifier_name
        classifier_comparison[classifier_name] = all_fold_metrics
        if noisy == 'quiet': 
            print()
            print(f"Average {metric_to_optimize}: {optimization_metric_avg:.3f}")
            print('-------------')
    # Return our results
    return best, best_name, classifier_comparison
    

#### **(a). Compare Naïve Bayes, Logistic Regression, and SVMs on a unigram feature space.**

In [ ]:
# consulted the example notebook 

# define X
vocab_size = 1000
# By default we can build a unigram model - capturing individual words only
vectorizer = CountVectorizer(max_features=vocab_size)
X = vectorizer.fit_transform(df_training["text"])

In [ ]:
# Now lets make a DataFrame with our BOW model representations for each message
bow_df = pd.DataFrame(X.toarray())
column_names = [str(i) for i in range(vocab_size)]
# Make the column names the words
for k, v in vectorizer.vocabulary_.items():
  column_names[v] = k
bow_df.columns = column_names
# Add our Y labels to our DataFrame
bow_df["labels_A"] = df_training["labels_A"].values
# Take a look at our DataFrame
bow_df

,00,00pm,04,0915,0ffsxzymap,10,100,11,11th,12,...,yfoa4i09ex,yoga,york,you,your,yrr3b,ytncrtupl2,yts,zachkleinwsb,labels_A
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Positive
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Negative
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Neutral
156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neutral
157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neutral
158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Positive


In [ ]:
# Pick Classifiers to Compare
from sklearn.svm import LinearSVC, SVC
classifiers = {
    "Bernoulli NB": BernoulliNB(),
    "Linear SVM": LinearSVC(),
    "RBF SVM": SVC(kernel='rbf'),
    "Poly SVM": SVC(kernel='poly'),
    "Complement NB": ComplementNB(), 
    "Multinomial NB": MultinomialNB(),
    "Logistic Regression": LogisticRegression(penalty="none", solver="lbfgs", multi_class='ovr', max_iter=10000, random_state=123),
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features
sorted_sentiments=["Negative", "Neutral", "Positive"]

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_A", labels=sorted_sentiments, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    


Bernoulli NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.021
-------------
Linear SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.093
-------------
RBF SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: -0.010
-------------
Poly SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.014
-------------
Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.037
-------------
Multinomial NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.063
-------------
Logistic Regression: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.120
-------------
Best classifier is: Logistic Regression 
With K=0.120.


Among Naïve Bayes, Logistic Regression, and SVMs, the **Logistic Regression** performs the best, and its kappa value eqauls 0.12.

#### **(b). Compare a unigram feature space with a feature space that also includes longer N-grams.**

In [ ]:
# Create a vocab of n-grams from the dataframe
def ngrams(df, vocab_size = 1000, max_n=1):
  vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=(1,max_n))
  X = vectorizer.fit_transform(df["text"])

  bow_df = pd.DataFrame(X.toarray())
  column_names = [str(i) for i in range(vocab_size)]
  for k, v in vectorizer.vocabulary_.items():
    column_names[v] = k
  bow_df.columns = column_names

  bow_df["labels_A"] = df["labels_A"].reset_index()['labels_A']
  return bow_df
# create unigram and bigrams DataFrame
unigram_df = ngrams(df_training, max_n=1)
bigram_df = ngrams(df_training, max_n=2)

In [ ]:
### Setup classifiers and metrics to be used on all n-grams
# Pick Classifiers to Compare
classifiers = {
     "Logistic Regression": LogisticRegression(penalty="none", solver="lbfgs", multi_class='ovr', max_iter=10000, random_state=123)
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

### Compare classifiers on unigrams ###
# Pick features to use
feature_set = list(unigram_df.columns[:-1])

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, unigram_df, feature_set, "labels_A", labels=sorted_sentiments, noisy = 'quiet')

print(f"The unigram classifier is: {best_name} \nWith K={best:.3f}.")    

### Compare classifiers on bigrams ###
# Pick features to use
feature_set = list(bigram_df.columns[:-1])

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bigram_df, feature_set, "labels_A", labels=sorted_sentiments, noisy = 'quiet')

print(f"The bigram classifier is: {best_name} \nWith K={best:.3f}.")    



Logistic Regression: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.120
-------------
The unigram classifier is: Logistic Regression 
With K=0.120.
Logistic Regression: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.117
-------------
The bigram classifier is: Logistic Regression 
With K=0.117.


Comparing unigram with bigram features using Logistic Regression classifier, **unigram** feature space performs slighlt better. The kappa value equals 0.12.

#### **(i). For Naive Bayes, evaluate different implementations: ComplementNB, MultinomialNB, BernoulliNB**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "Bernoulli NB": BernoulliNB(),
    "Complement NB": ComplementNB(), 
    "Multinomial NB": MultinomialNB()
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_A", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

Bernoulli NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.021
-------------
Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.037
-------------
Multinomial NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.063
-------------
Best classifier is: Multinomial NB 
With K=0.063.


Among three types of Naive Bayes classifers, **Multinomial NB** performs the best. The best kappa value is 0.063.

#### **(k). For Logistic Regression, try L1 and L2 regularization, as well as unregularized features.**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "LogisticRegression l2": LogisticRegression(penalty="l2", solver="saga", max_iter=10000, random_state=123),
    "LogisticRegression l1": LogisticRegression(penalty='l1', random_state=123, solver='saga', max_iter=10000), 
    "Unregularized Features": LogisticRegression(random_state=123, solver='saga', max_iter=10000)}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_A", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

LogisticRegression l2: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.121
-------------
LogisticRegression l1: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.161
-------------
Unregularized Features: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.121
-------------
Best classifier is: LogisticRegression l1 
With K=0.161.


Based on the above evaluation, logistic regression models with l1 penalty has the highest kappa value, which is 0.161. Thus the model with **l1 penalty** performs better. 

#### **(j). For Support Vector Machines, evaluate different kernels including a polynomial kernel and a radial basis function kernel.**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "Linear SVM": LinearSVC(),
    "RBF SVM": SVC(kernel='rbf'),
    "Poly SVM": SVC(kernel='poly')
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_A", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

Linear SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.093
-------------
RBF SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: -0.010
-------------
Poly SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.014
-------------
Best classifier is: Linear SVM 
With K=0.093.


Based on the result, **Linear SVM** performs the best, with a kappa value of 0.093.

## Conclusion
Based on the five optimizations above, **LogisticRegression(max_iter=10000, penalty='l1', random_state=123,
                   solver='saga') on unigram feature face** is the best model, with Kappa value of 0.161. 

# Task 2.3

Train two models, each **trained on the full 80% training set, and tested on the held-out 20% test set**:
   - A Naïve Bayes classifier with unigram features.
   - The best-tuned model from task two, retrained on the full 80% training set. 
   
Report three sets of evaluation metrics:
   - The estimated performance from cross-validation in Task 2.
   - The performance of the simple Naïve Bayes unigram classifier on the held-out test set.
   - The performance of the best-tuned model on the held-out test set.

At minimum, your evaluations should include percent accuracy and Kappa values.

####**A Naive Bayes classifier with unigram features**

In [ ]:
# Train a Naive Bayes classifier with unigram features.
# Here we choose BernoulliNB() as our Naive Bayes classifer.
unigram_df=ngrams(df, max_n=1)
feature_set = list(unigram_df.columns[:-1])
unigram_training=unigram_df.iloc[df_training.index,:]
unigram_testing=unigram_df.iloc[df_testing.index,:]

X_train=unigram_training.loc[:, feature_set]
y_train=unigram_training.iloc[:,-1]
X_test=unigram_testing.loc[:, feature_set]
y_test=unigram_testing.iloc[:,-1]
NB=BernoulliNB()
NB.fit(X_train,y_train)


BernoulliNB()

####**The best-tuned model from task two, retrained on the full 80% training set.**

In [ ]:
# Train the best performance model- Logistic Regression unigram features
lr=LogisticRegression(penalty='l1', random_state=123, solver='saga', max_iter=10000)
lr.fit(X_train,y_train)


LogisticRegression(max_iter=10000, penalty='l1', random_state=123,
                   solver='saga')

####**Report three sets of evaluation metrics:**
  

In [ ]:
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
# Calculate and print the estimated performance from 10-fold-validation.
a_scores = cross_val_score(estimator = NB,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = "accuracy")
# print('10-fold accuracy: %s' % a_scores)
print("Estimated performance from cross-validation:")
print('10-fold accuracy: %.3f +/- %.3f' % (np.mean(a_scores), np.std(a_scores)))

kappa_scorer = make_scorer(cohen_kappa_score)
k_scores = cross_val_score(estimator = NB,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = kappa_scorer)
# print('10-fold Cohen's Kappa: %s' % k_scores)
print('10-fold Kappa: %.3f +/- %.3f' % (np.mean(k_scores), np.std(k_scores)))
print("------------------------------------")

# Calculate and print the performance of the Bernoulli NB model on the held-out test set.
print("Performance on the held-out test set:")
y_pred = NB.predict(X_test)
accuracy_NB = accuracy_score(y_test, y_pred)
print(f"The accuracy is: {accuracy_NB:.4f}.")
kappa_NB = cohen_kappa_score(y_test, y_pred)
print(f"The Kappa is: {kappa_NB:.4f}.")

Estimated performance from cross-validation:
10-fold accuracy: 0.506 +/- 0.052
10-fold Kappa: 0.021 +/- 0.097
------------------------------------
Performance on the held-out test set:
The accuracy is: 0.6000.
The Kappa is: 0.1011.


In [ ]:
# Calculate and print the estimated performance from 10-fold-validation.
a_scores2 = cross_val_score(estimator = lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = "accuracy")
# print('10-fold accuracy: %s' % a_scores)
print("Estimated performance from cross-validation:")
print('10-fold accuracy: %.3f +/- %.3f' % (np.mean(a_scores2), np.std(a_scores2)))

kappa_scorer = make_scorer(cohen_kappa_score)
k_scores2 = cross_val_score(estimator = lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = kappa_scorer)
# print('10-fold Cohen's Kappa: %s' % k_scores)
print('10-fold Kappa: %.3f +/- %.3f' % (np.mean(k_scores2), np.std(k_scores2)))
print("------------------------------------")

# Calculate and print the performance of the best tuned model on the held-out test set.
print("Performance on the held-out test set:")
y_pred = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred)
print(f"The accuracy is: {accuracy_lr:.4f}.")
kappa_lr = cohen_kappa_score(y_test, y_pred)
print(f"The Kappa is: {kappa_lr:.4f}.")

Estimated performance from cross-validation:
10-fold accuracy: 0.525 +/- 0.116
10-fold Kappa: 0.149 +/- 0.203
------------------------------------
Performance on the held-out test set:
The accuracy is: 0.5250.
The Kappa is: 0.1017.


Based on the three sets of evaluation metircs above, the accuracy for simple Naïve Bayes unigram classifier on the held-out test set is 0.6 and Kappa is 0.1011, while the estimated performance from 10-fold cross-validation in task 2 shows accuracy 0.506 +/- 0.052 and Kappa 0.021 +/- 0.097. The accuracy is higher than the estimated range, and Kappa is in the range. 

For the best-tuned model (Logistic Regression model) on the held-out test set, the accuracy is 0.5250 and Kappa is 0.1017, while the estimated performance from 10-fold cross-validation in task 2 shows accuracy 0.525 +/- 0.116 and Kappa 0.149 +/- 0.203. Both the accuracy and Kappa are in the estimated range. 

The Kappa value is slightly better with the best-tuned model than the one with Naïve Bayes unigram classifier and accuracy score is the other way around. 

#1. Extra Credit

For up to 2 points of extra credit:
   - Perform five of the optimizations above instead of three.


(Please see code above).

#2. Extra Credit

For up to 2 points of extra credit:
   - Re-run the same set of code above but on `labels_B` instead of `labels_A`. Choose a new set of optimized hyperparameters and features based on these results. How do the chosen models differ, and are the model quality metrics different between the two columns in a statistically significant way?

#### **(a). Compare Naïve Bayes, Logistic Regression, and SVMs on a unigram feature space. (Labels_B)**

In [ ]:
vocab_size = 1000
# By default we can build a unigram model - capturing individual words only
vectorizer = CountVectorizer(max_features=vocab_size)
X = vectorizer.fit_transform(df_training["text"])
# Now lets make a DataFrame with our BOW model representations for each message
bow_df = pd.DataFrame(X.toarray())
column_names = [str(i) for i in range(vocab_size)]
# Make the column names the words
for k, v in vectorizer.vocabulary_.items():
  column_names[v] = k
bow_df.columns = column_names
# Add our Y labels to our DataFrame
bow_df["labels_B"] = df_training["labels_B"].values
# Take a look at our DataFrame
bow_df

,00,00pm,04,0915,0ffsxzymap,10,100,11,11th,12,...,yfoa4i09ex,yoga,york,you,your,yrr3b,ytncrtupl2,yts,zachkleinwsb,labels_B
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neutral
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Positive
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Negative
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Neutral
156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neutral
157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neutral
158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Positive


In [ ]:
# Pick Classifiers to Compare
from sklearn.svm import LinearSVC, SVC
classifiers = {
    "Bernoulli NB": BernoulliNB(),
    "Linear SVM": LinearSVC(),
    "RBF SVM": SVC(kernel='rbf'),
    "Poly SVM": SVC(kernel='poly'),
    "Complement NB": ComplementNB(), 
    "Multinomial NB": MultinomialNB(),
    "Logistic Regression": LogisticRegression(penalty="none", solver="lbfgs", multi_class='ovr', max_iter=10000, random_state=123),
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features
sorted_sentiments=["Negative", "Neutral", "Positive"]

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_B", labels=sorted_sentiments, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

Bernoulli NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.058
-------------
Linear SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.105
-------------
RBF SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: -0.035
-------------
Poly SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.017
-------------
Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.152
-------------
Multinomial NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.136
-------------
Logistic Regression: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.114
-------------
Best classifier is: Complement NB 
With K=0.152.


Among Naïve Bayes, Logistic Regression, and SVMs, the **Completement NB** performs the best, its kappa value eqauls 0.152.

#### **(b). Compare a unigram feature space with a feature space that also includes longer N-grams. (Labels_B)**

In [ ]:
# Create a vocab of n-grams from the dataframe
def ngrams(df, vocab_size = 1000, max_n=1):
  vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=(1,max_n))
  X = vectorizer.fit_transform(df["text"])

  bow_df = pd.DataFrame(X.toarray())
  column_names = [str(i) for i in range(vocab_size)]
  for k, v in vectorizer.vocabulary_.items():
    column_names[v] = k
  bow_df.columns = column_names

  bow_df["labels_B"] = df["labels_B"].reset_index()['labels_B']
  return bow_df
# create unigram and bigrams DataFrame
unigram_df = ngrams(df_training, max_n=1)
bigram_df = ngrams(df_training, max_n=2)

In [ ]:
### Setup classifiers and metrics to be used on all n-grams
# Since Complement NB performs the best in a), we pick it as our Classifier
classifiers = {
    "Complement NB": ComplementNB()
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

### Compare classifiers on unigrams ###
# Pick features to use
feature_set = list(unigram_df.columns[:-1])

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, unigram_df, feature_set, "labels_B", labels=sorted_sentiments, noisy = 'quiet',)

print(f"The unigram classifier is: {best_name} \nWith K={best:.3f}.")    

### Compare classifiers on bigrams ###
# Pick features to use
feature_set = list(bigram_df.columns[:-1])

# Compare models and display final result
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bigram_df, feature_set, "labels_B", labels=sorted_sentiments, noisy = 'quiet',)

print(f"The bigram classifier is: {best_name} \nWith K={best:.3f}.")    

Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.152
-------------
The unigram classifier is: Complement NB 
With K=0.152.
Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.158
-------------
The bigram classifier is: Complement NB 
With K=0.158.


Comparing unigram with bigram using Complement NB classifier, **bigram feature space** performs slighlt better. The kappa value equals 0.158.

#### **(i). For Naive Bayes, evaluate different implementations: ComplementNB, MultinomialNB, BernoulliNB. (Labels_B)**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "Bernoulli NB": BernoulliNB(),
    "Complement NB": ComplementNB(), 
    "Multinomial NB": MultinomialNB()
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_B", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

Bernoulli NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.058
-------------
Complement NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.152
-------------
Multinomial NB: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.136
-------------
Best classifier is: Complement NB 
With K=0.152.


Among three types of Naive Bayes classifers, **Complement NB** performs the best. The best kappa value is 0.152.


#### **(k). For Logistic Regression, try L1 and L2 regularization, as well as unregularized features. (Labels_B)**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "LogisticRegression l2": LogisticRegression(penalty="l2", solver="saga", max_iter=10000, random_state=123),
    "LogisticRegression l1": LogisticRegression(penalty='l1', random_state=123, solver='saga', max_iter=10000), 
    "Unregularized Features": LogisticRegression(random_state=123, solver='saga', max_iter=10000)}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_B", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

LogisticRegression l2: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.118
-------------
LogisticRegression l1: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.049
-------------
Unregularized Features: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.118
-------------
Best classifier is: LogisticRegression l2 
With K=0.118.


Based on the above evaluation, logistic regression models with l2 penalty and unregularized logistic regression have similiar kappa value and both greately higher than logistic regression model with l1 pental. Logistic regression model with l2 penalty has the slight higher kappa value, which is 0.118. Thus the model with **l2 penalty** performs better. 

#### **(j). For Support Vector Machines, evaluate different kernels including a polynomial kernel and a radial basis function kernel. (Labels_B)**

In [ ]:
# Pick Classifiers to Compare
classifiers = {
    "Linear SVM": LinearSVC(),
    "RBF SVM": SVC(kernel='rbf'),
    "Poly SVM": SVC(kernel='poly')
}

# Set a list of metrics we want to use to compare our classifiers 
metrics = {
    "Accuracy" : lambda y,y_pred: 100*accuracy_score(y,y_pred),
    "Kappa"    : cohen_kappa_score
}

# Choose a metric to optimize over
metric_to_optimize = 'Kappa'

# Pick features to use
bow_features = column_names
feature_set = bow_features

# Compare models and display final result
sorted_labels = ["Negative", "Neutral", "Positive"]
best, best_name, classifier_comparison = compare_classifiers(classifiers, metrics, metric_to_optimize, bow_df, feature_set, "labels_B", labels=sorted_labels, noisy = 'quiet',)

print(f"Best classifier is: {best_name} \nWith K={best:.3f}.")    

Linear SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.105
-------------
RBF SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: -0.035
-------------
Poly SVM: Fold 0...1...2...3...4...5...6...7...8...9
Average Kappa: 0.017
-------------
Best classifier is: Linear SVM 
With K=0.105.


Based on the result, **Linear SVM** performs the best, with a kappa value of 0.105.

## Conclusion
Among the five optimizations above, the best model is the one with **ComplementNB model with bigram feature space**. 

# Evaluation

####**A Naive Bayes classifier with unigram features (Labels_B)**

In [ ]:
# Train a Naive Bayes classifier with unigram features.
unigram_df=ngrams(df, max_n=1)
feature_set = list(unigram_df.columns[:-1])
unigram_training=unigram_df.iloc[df_training.index,:]
unigram_testing=unigram_df.iloc[df_testing.index,:]
X_train=unigram_training.loc[:, feature_set]
y_train=unigram_training.iloc[:,-1]
X_test=unigram_testing.loc[:, feature_set]
y_test=unigram_testing.iloc[:,-1]
NB=BernoulliNB()
NB.fit(X_train,y_train)

BernoulliNB()

####**The best-tuned model from task two, retrained on the full 80% training set. (Labels_B)**



In [ ]:
# Train the best performance model-Complement NB bigram features
NBC=ComplementNB()
bigram_df=ngrams(df, max_n=2)
feature_set = list(bigram_df.columns[:-1])
bigram_training=bigram_df.iloc[df_training.index,:]
bigram_testing=bigram_df.iloc[df_testing.index,:]
X_train2=bigram_training.loc[:, feature_set]
y_train2=bigram_training.iloc[:,-1]
X_test2=bigram_testing.loc[:, feature_set]
y_test2=bigram_testing.iloc[:,-1]
NBC.fit(X_train2,y_train2)

ComplementNB()

####**Report three sets of evaluation metrics (Labels_B):**

In [ ]:
# Calculate and print the estimated performance from 10-fold-validation.
a_scores = cross_val_score(estimator = NB,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = "accuracy")
# print('10-fold accuracy: %s' % a_scores)
print("Estimated performance from cross-validation:")
print('10-fold accuracy: %.3f +/- %.3f' % (np.mean(a_scores), np.std(a_scores)))

kappa_scorer = make_scorer(cohen_kappa_score)
k_scores = cross_val_score(estimator = NB,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1,
                         scoring = kappa_scorer)
# print('10-fold Cohen's Kappa: %s' % k_scores)
print('10-fold Kappa: %.3f +/- %.3f' % (np.mean(k_scores), np.std(k_scores)))
print("------------------------------------")

# Calculate and print the performance of the Bernoulli NB model on the held-out test set.
print("Performance on the held-out test set:")
y_pred = NB.predict(X_test)
accuracy_NB = accuracy_score(y_test, y_pred)
print(f"The accuracy is: {accuracy_NB:.4f}.")
kappa_NB = cohen_kappa_score(y_test, y_pred)
print(f"The Kappa is: {kappa_NB:.4f}.")

Estimated performance from cross-validation:
10-fold accuracy: 0.481 +/- 0.101
10-fold Kappa: 0.096 +/- 0.171
------------------------------------
Performance on the held-out test set:
The accuracy is: 0.5250.
The Kappa is: 0.2475.


In [ ]:
# Calculate and print the estimated performance from 10-fold-validation.
a_scores2 = cross_val_score(estimator = NBC,
                         X = X_train2,
                         y = y_train2,
                         cv = 10,
                         n_jobs = 1,
                         scoring = "accuracy")
# print('10-fold accuracy: %s' % a_scores)
print("Estimated performance from cross-validation:")
print('10-fold accuracy: %.3f +/- %.3f' % (np.mean(a_scores2), np.std(a_scores2)))

kappa_scorer = make_scorer(cohen_kappa_score)
k_scores2 = cross_val_score(estimator = NBC,
                         X = X_train2,
                         y = y_train2,
                         cv = 10,
                         n_jobs = 1,
                         scoring = kappa_scorer)
# print('10-fold Cohen's Kappa: %s' % k_scores)
print('10-fold Kappa: %.3f +/- %.3f' % (np.mean(k_scores2), np.std(k_scores2)))
print("------------------------------------")

# Calculate and print the performance of the best-tuned model on the held-out test set.
print("Performance on the held-out test set:")
y_pred = NBC.predict(X_test2)
accuracy_NBC = accuracy_score(y_test2, y_pred)
print(f"The accuracy is: {accuracy_NBC:.4f}.")
kappa_NBC = cohen_kappa_score(y_test2, y_pred)
print(f"The Kappa is: {kappa_NBC:.4f}.")

Estimated performance from cross-validation:
10-fold accuracy: 0.456 +/- 0.097
10-fold Kappa: 0.163 +/- 0.145
------------------------------------
Performance on the held-out test set:
The accuracy is: 0.4750.
The Kappa is: 0.1977.


Based on the three sets of evaluation metircs above, the accuracy for simple Naïve Bayes unigram classifier on the held-out test set is 0.525 and Kappa is 0.2475, while the estimated performance from 10-fold cross-validation in task 2 shows accuracy 0.481 +/- 0.101 and Kappa 0.096 +/- 0.171. Both the accuracy and Kappa are in the estimated range. 

For the best-tuned model (ComplementNB with bigram) on the held-out test set, the accuracy is 0.4750 and Kappa is 0.1977, while the estimated performance from 10-fold cross-validation in task 2 shows accuracy 0.456 +/- 0.097 and Kappa 0.163 +/- 0.145. Both the accuracy and Kappa are in the estimated range. 


**Comparison between labels_A and labels_B:** the best-tuned model with labels_A is logistic regression on unigram feature space, while with labels_B is the one with the Complement Naive Bayes classifier on bigram feature space. Comparing the best-tuned model of both labels_A and labels_B, we mainly look at the accuracy and kappa value on the held-out test set. For accuracy, labels_A is 0.5250 and labels_B is 0.4750, so labels_A is 0.05 better than labels_B. For Kappa, labels_A is 0.1017 and labels_B is 0.1977, so labels_B is 0.096 better than labels_A. Since mainly we rely on Kappa score for judgement, the performance with **labels_B is better** overall.  


# Scoring Rubric
![](https://drive.google.com/uc?export=view&id=1JqI8Tfmi3YrnjVdDxjwNu1ZOuOOgQCuI)

![](https://drive.google.com/uc?export=view&id=1VfVuKGmNBu6oJgXBTX4YB6Lxe_0t9cWN)